### Links

#### Gurobi AI Modeling Documentation
https://gurobi-optimization-gurobi-ai-modeling.readthedocs-hosted.com/en/latest/

#### Gurobi AI Modeling GPT
https://chatgpt.com/g/g-g69cy3XAp-gurobi-ai-modeling-assistant

This will work best if you have a *Plus* account, but should generate formulations and code without it. 

#### Source Data
https://www.kaggle.com/datasets/pedroisrael/mix-marketing-analysis?select=Bathrooms_random_data2.csv

### Prompt Template

Problem description:
<describe the problem on a high level, making sure you also introduce the context. Normally, 3-5 sentences is sufficient for this.>

Objective: <Maximize/Minimize> <objective>

Constraints:
- <constraint name 1>: <describe constraint>
- <constraint name 2>: <describe constraint>
- <constraint name 3>: <describe constraint>
- etc.

Data:
<data in csv format, including headers> or \<upload your files and name the filename here>

### Regression Data

In [1]:
import pandas as pd
path = 'https://raw.githubusercontent.com/yurchisin/ODSC2024/refs/heads/main/data_files/'
lm_coeff = pd.read_csv(path + 'lm_coeff.csv', index_col=['productcategory','mediatype','platform'])
lm_coeff

slope  min_spend  \
productcategory mediatype            platform                                 
Other           Social               Pinterest           91.6273        0.0   
                                     Facebook            82.2761        0.0   
                Bing Search          Bing                 3.3551        1.0   
                Programmatic Video   discovery           36.5491        0.0   
                Programmatic Display discovery           27.4809        0.0   
                                     padsquad            68.5564       11.0   
                                     the trade desk     220.2158        0.0   
                                     kargo              124.7605        1.0   
Toilets         Social               Pinterest           69.8756        0.0   
                                     Facebook           124.2930        4.0   
                Bing Search          Bing               123.4833        0.0   
                Direct Display       new york times_pg   51.9730        0.0   
                                     uncrate             50.0850        0.0   
                OOH                  big outdoor         65.7552      846.7   
                                     clear channel       57.7892    49888.8   
                National TV          discovery          192.1910    58983.4   
Multiple        Social               Facebook           105.5983        3.0   
                Bing Search          Bing                 5.8760        0.0   
Bath Sinks      Social               Facebook           257.5944        1.0   
Bath Faucets    Social               Pinterest          141.7047        0.0   
                                     Facebook           136.3193        2.0   
Kitchen Sinks   Social               Facebook           267.8804        1.0   
                Bing Search          Bing                57.1451        0.0   
Showers/Baths   Social               Pinterest          162.3006        0.0   
                                     Facebook           184.1421        3.0   
                Bing Search          Bing               105.9903        1.0   
                Direct Display       discovery           70.6940       12.0   
                National TV          scatter            215.7597    60298.8   
                Social               TikTok             221.9989       21.0   
                                     Reddit             174.2360        9.0   
                Direct Video         hulu.com            26.0803       29.0   
                                     hbomax.com          13.6221       19.0   
                                     lg - us             25.8478        0.0   
Kitchen Faucets Social               Pinterest          187.7129        0.0   
                                     Facebook           217.7424        1.0   

                                                        max_spend      n  
productcategory mediatype            platform                             
Other           Social               Pinterest                474   9646  
                                     Facebook               10909  10796  
                Bing Search          Bing                     541   5219  
                Programmatic Video   discovery               2142   1692  
                Programmatic Display discovery              22421     52  
                                     padsquad                3729   2534  
                                     the trade desk          4145   1575  
                                     kargo                    564   1060  
Toilets         Social               Pinterest               1252   5596  
                                     Facebook                3983  18698  
                Bing Search          Bing                    1632   6526  
                Direct Display       new york times_pg       5192   3477  
                                     uncrate                  957    411  
          

In [ ]:
### Paste any code you want to test out
# %pip install gurobipy

### How I'd Model the Problem:
Adding in an assumption that there are no free ads

In [2]:
%pip install gurobipy

import gurobipy as gp
from gurobipy import GRB

m = gp.Model("mmo")

no_free_ads = 10 # assume there is a positive min spend, calculate this how ever you want
B = 373896 # average total weekly spend, use that as budget

spend = m.addVars(lm_coeff.index, vtype=GRB.SEMICONT, lb = (lm_coeff.min_spend+no_free_ads), ub = (lm_coeff.max_spend+no_free_ads), name = 'spend')
impressions = m.addVars(lm_coeff.index, name = 'impressions')

lm_eqs = m.addConstrs((impressions[i,j,k] == lm_coeff.slope[i,j,k]*spend[i,j,k] for [i, j, k] in lm_coeff.index), name='lm_eqs')
total_budget = m.addConstr(spend.sum() <= B, name = 'total_budget')

m.setObjective(impressions.sum(), sense = GRB.MAXIMIZE)

m.optimize()

spend_values = pd.Series(m.getAttr('X', spend), name = "dollars", index = lm_coeff.index)
spend_values[spend_values > 0]

Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[rosetta2] - Darwin 23.6.0 23G80)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 36 rows, 70 columns and 105 nonzeros
Model fingerprint: 0x0a65bb6d
Variable types: 35 continuous, 0 integer (0 binary)
Semi-Variable types: 35 continuous, 0 integer
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+01, 1e+05]
  RHS range        [4e+05, 4e+05]
Presolve removed 35 rows and 35 columns
Presolve time: 0.00s
Presolved: 69 rows, 70 columns, 171 nonzeros
Presolved model has 2 SOS constraint(s)
Variable types: 35 continuous, 35 integer (35 binary)
Found heuristic solution: objective 5.233005e+07

Root relaxation: objective 6.439115e+07, 24 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds  

productcategory  mediatype             platform      
Other            Social                Pinterest            484.0
                                       Facebook           10919.0
                 Programmatic Display  the trade desk      4155.0
                                       kargo                574.0
Toilets          Social                Facebook            3993.0
                 Bing Search           Bing                1642.0
                 OOH                   clear channel      49898.8
                 National TV           discovery         113465.0
Multiple         Social                Facebook           11927.0
Bath Sinks       Social                Facebook             732.0
Bath Faucets     Social                Pinterest            191.0
                                       Facebook             475.0
Kitchen Sinks    Social                Facebook             353.0
Showers/Baths    Social                Pinterest           5537.0
                      